In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl
import seaborn as sns

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
df= pd.read_csv('train.csv')

In [3]:
df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,697,705,Asus,Chromebook Flip,2 in 1 Convertible,12.5,Full HD / Touchscreen 1920x1080,Intel Core M M3-6Y30 0.9GHz,4GB,64GB Flash Storage,Intel HD Graphics 515,Chrome OS,1.2kg,669.0
1,435,442,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1600 3.2GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1695.0
2,735,743,Lenovo,V310-15IKB (i7-7500U/4GB/1TB/FHD/W10),Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,4GB,1TB HDD,Intel HD Graphics 620,Windows 10,1.85kg,779.0
3,864,875,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7660U 2.5GHz,16GB,512GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.29kg,2240.0
4,1176,1194,Lenovo,B51-80 (i7-6500U/4GB/1008GB/FHD/W7),Notebook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,1.0TB Hybrid,Intel HD Graphics 520,Windows 7,2.32kg,825.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

import warnings as wr

wr.filterwarnings('ignore')


# Cargamos datos
df = pd.read_csv('train.csv')

# Preprocesamos y creamos de variables dummy
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df = pd.get_dummies(df, columns=['Company', 'Product', 'TypeName', 'ScreenResolution', 'OpSys', 'Cpu', 'Gpu', 'Memory'])

# Separamos en características y objetivo
X = df.drop(columns=['Price_euros', 'id'])
y = df['Price_euros']

# Dividimos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalamos las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Modelos base con hiperparámetros ajustados
estimators = [
    ('rf', RandomForestRegressor(n_estimators=3, max_depth=5, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=5000, max_depth=7, learning_rate=0.1, random_state=42)),
    ('elasticnet', ElasticNet(alpha=0.5, l1_ratio=1, selection ="cyclic", random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=250,learning_rate=0.4, max_depth=8, random_state=42))
]

# Modelo de stacking con meta-modelo diferente
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=Lasso(alpha=1.5), # puedo meter Ridge también
    cv=5
)

stacking_regressor.fit(X_train_scaled, y_train)
y_pred = stacking_regressor.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print(f'Stacking Regressor MAE: {mae}')

# Cargamos y preprocesamos el conjunto de prueba
test_df = pd.read_csv('test.csv')
test_ids = test_df['id']  # Guardar los IDs para usarlos después
test_df['Ram'] = test_df['Ram'].str.replace('GB', '').astype(int)
test_df['Weight'] = test_df['Weight'].str.replace('kg', '').astype(float)
test_df = pd.get_dummies(test_df, columns=['Company', 'Product', 'TypeName', 'ScreenResolution', 'OpSys', 'Cpu', 'Gpu', 'Memory'])

# Alineamos las columnas con el conjunto de entrenamiento
test_df = test_df.reindex(columns=df.columns, fill_value=0).drop(columns=['Price_euros', 'id'])

# Escalamos las características del conjunto de prueba
X_test_scaled = scaler.transform(test_df)

# Realizamos predicciones en el conjunto de prueba
y_pred_test = stacking_regressor.predict(X_test_scaled)

# Creamos un DataFrame con los IDs y las predicciones
predictions_df = pd.DataFrame({
    'id': test_ids,
    'Price_euros': y_pred_test
})

# Guardamos el DataFrame como un archivo CSV
# predictions_df.to_csv('submission_daniel_egoitz_FINAL2.csv', index=False)

Stacking Regressor MAE: 146.81050241033694
